In [6]:
import math

In [7]:
test = [2.8533979930661495, 0.8350593062280169, 2.9865046080782, 1.8350735010944388, 1.2960796257017295, 1.8479314037072352]

In [8]:
class BaseSection:
    def __init__(self, angle , type):
        self.angle = angle
        self.an_normalized = self.angle_normalized()

    def angle_normalized(self):
        if self.angle > math.pi/2:
            ang = math.pi - self.angle
        else:
            ang = self.angle
        return ang


In [9]:
for i in test:
    i_b = BaseSection(i)
    print(i_b.an_normalized)

0.2881946605236436
0.8350593062280169
0.15508804551159328
1.3065191524953543
1.2960796257017295
1.293661249882558
